# PWC Scraper

This scraper collects texts from the "Artificial Intelligence" subsection of PWC.com: https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html

## Libraries

Loading libraries...

In [21]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import re

import time

## URL List
The scraper collects urls from the insights on the "artificial intelligence" subsection of pwc.com: https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html

In [22]:
start_url = 'https://www.pwc.com/gx/en/issues/data-and-analytics/artificial-intelligence.html'

In [23]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('button#acceptCookie.btn.cookie-accept').click()
time.sleep(2)

y_coord = 4000
coordinates = {'x': 0, 'y': y_coord}
driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
time.sleep(3)

while True:
    try:
        driver.find_element_by_css_selector("a.btn.btn--transparent.collection__load-more.ng-binding.ng-scope").click()
        time.sleep(3)
        y_coord += 1000
        coordinates = {'x': 0, 'y': y_coord}
        driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
        time.sleep(3)
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No more to load")
        break

pageSource = driver.page_source

driver.quit()

No more to load


In [24]:
sel = Selector(text = pageSource) #create Selector object
urldict = {}
   
urldict['titles'] = sel.css('a.collection__item-link > div.collection__item-content > h4.regular.collection__item-heading > span::text').extract()
urldict['urls'] = sel.css('a.collection__item-link::attr(href)').extract()

In [25]:
print(len(urldict['titles']),
     len(urldict['urls']))

100 100


In [26]:
urllist = urldict['urls']

In [27]:
#URL dump
with open('../data_raw/pwc_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

In [28]:
# Load URLs
urllist = []
with open('../data_raw/pwc_urls.txt', 'r') as f:
    for line in f:
        urllist.append(line.strip())

print(len(urllist),
      '\n',
     urllist[0])

100 
 https://www.pwc.co.uk/services/sustainability-climate-change/insights/new-nature-economy-nature-risk-rising.html
